In [1]:
# 

In [4]:
import numpy as np
import pandas as pd

In [5]:
trip = pd.read_csv(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Export\Release 3\General Release\Unzipped\2014-pr3-hhsurvey-trips.csv')

C:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\pandas\io\parsers.py:1159: DtypeWarning: Columns (6,56,57,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [49]:
person = pd.read_csv(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Export\Release 3\General Release\Unzipped\2014-pr3-hhsurvey-persons.csv')

C:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\pandas\io\parsers.py:1159: DtypeWarning: Columns (107) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [9]:
# Filter out values with negative reported trip times
trip = trip.query('trip_dur_reported > 0')

In [10]:
expwt = 'expwt_final'

In [40]:
# Information on the average distance covered and average time spent in the different modes 
# (especially car), e.g., per day.

In [33]:
# Create field that is the weighted distance and time
trip['wt_gdist'] = trip.gdist * trip[expwt]
trip['wt_gtime'] = trip.gtime * trip[expwt]
trip['wt_reported_time'] = trip.trip_dur_reported * trip[expwt]

In [53]:
# Total persons in region
total_pop = person[expwt].sum()

In [212]:
total_pop

3464535.644400008

In [184]:
# Count of all people using mode at least once

# list of trips by mode by person
trips_by_mode = pd.pivot_table(data=trip, rows='personID', columns='mode', aggfunc='count', values=expwt, fill_value=0)

# Replace the sum with a 1 for indicator of use of this mode in the day
indicator_df = trips_by_mode[[i for i in range(1,18)]].applymap(lambda x: 1 if x > 0 else 0)

indicator_df['personid'] = indicator_df.index

# Join the list of persons to the original person file to get expansion weight
df = pd.merge(indicator_df,person[['personid',expwt]],on='personid')

In [191]:
# Get total number of people using each mode per day (for one or more trips per day)
# Sum the expansion weight times the indicator for each person 
my_dict = {}
for i in range(1,18):
    my_dict[i] = (df[i]*df[expwt]).sum()

In [203]:
# Dictionary to dataframe
results = pd.DataFrame.from_dict(my_dict.values())
results.index=my_dict.keys()
results.to_clipboard()

-------------------

In [206]:
# Distances covered in miles by individual persons per day
# For all trips by this mode
total_dist_by_mode = pd.pivot_table(data=trip, rows='personID', columns='mode', aggfunc='sum', values='wt_gdist', fill_value=0).sum()

In [ ]:
# Need a count of all the people using each mode at least once a day 

In [208]:
# Time in vehicle by individual persons per day
mean_time_by_mode = pd.pivot_table(data=trip, rows='personID', columns='mode', aggfunc='sum', values='wt_gtime', fill_value=0).sum()/total_pop
total_time_by_mode = pd.pivot_table(data=trip, rows='personID', columns='mode', aggfunc='sum', values='wt_gtime', fill_value=0).sum()


In [210]:
# Time in vehicle by individual persons per day
mean_time_by_mode_reported = pd.pivot_table(data=trip, rows='personID', columns='mode', aggfunc='sum', values='wt_reported_time', fill_value=0).sum()/total_pop
total_time_by_mode_reported = pd.pivot_table(data=trip, rows='personID', columns='mode', aggfunc='sum', values='wt_reported_time', fill_value=0).sum()
# total_dist_by_mode

In [211]:
total_time_by_mode_reported.to_clipboard()

In [44]:
df = data.trip

# Aggregate modes
hbw = df.query('d_purpose == 2 and o_purpose == 1')
hbo = df.query('o_purpose == 1 and d_purpose != 2')
nhb = df.query('o_purpose != 1')

In [50]:
df = hbo

# Average trip DURATION by mode
mode_group = df.groupby('mode').sum()
mode_group.wt_avg = mode_group['wt_reported_time']/df.groupby('mode')[expwt].sum()

# Final trip length by mode table
mode_group.wt_avg.to_clipboard()

In [51]:
mode_group[expwt].to_clipboard()

In [20]:
# Average trip time by purpose
mode_group.wt_avg_time_purp = mode_group['wt_gtime']/data.trip.groupby('d_purpose').expwt_2.sum()

# Final trip length by pur table
mode_group.wt_avg_time_purp.to_clipboard()

In [13]:
# Average trip time by county
cnty_group = data.trip.groupby('dcnty').sum()
mode_group.wt_avg_time_purp = mode_group['wt_gtime']/data.trip.groupby('dcnty').expwt_2.sum()

# Final trip length by pur table
mode_group.wt_avg_time_purp.to_clipboard()